# Variables

In [566]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [567]:
import sys
sys.path.append('..')
print(sys.path)
from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
import pprint

['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/mnt/g/Ai agents course/LangGraph/venv/lib/python3.12/site-packages', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..', '..']


# Testing

In [568]:
df_task = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [569]:
df_task

,STT,Task,Validation data
0,1,"""Bối cảnh của vấn đề:\nNhận dạng chữ viết tay ...",Link
1,2,Bối cảnh của vấn đề:\nTrong thời đại thông tin...,Link
2,3,Bối cảnh của vấn đề:\nĐây là một nhiệm vụ hồi ...,Link
3,4,Bối cảnh của vấn đề:\nĐiều quan trọng là các c...,Link
4,5,Bối cảnh của vấn đề:\nTrong nhiều ứng dụng thự...,Link


In [570]:
task = df_task["Task"][0]

In [571]:
task

'"Bối cảnh của vấn đề:\nNhận dạng chữ viết tay là một bài toán cơ bản trong lĩnh vực học máy và xử lý ảnh, với nhiều ứng dụng thực tiễn như nhận dạng chữ số trên phiếu khảo sát, hóa đơn hay bài thi tự động. Việc phân loại chữ số viết tay thành số nguyên tố hoặc không giúp mở rộng khả năng ứng dụng trong các bài toán toán học tự động, kiểm tra bài tập, hoặc các ứng dụng giáo dục.\n\nYêu cầu cụ thể cần đạt được:\nXây dựng một hệ thống có khả năng nhận diện chữ số viết tay từ ảnh và xác định xem số đó có phải là số nguyên tố hay không.\nĐầu vào là ảnh chứa một chữ số viết tay.\nĐầu ra là nhãn phân loại nhị phân: ""nguyên tố"" hoặc ""không nguyên tố"".\n\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\nMột thư mục có tên là ""images"" chứa các ảnh grayscale 28x28 pixel, mỗi ảnh chứa một chữ số viết tay từ 0 đến 9.\n\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\ncó các cột:\nfile_name: tên file ảnh\nprediction: nhãn của ản

In [572]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [573]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    # scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    # scrape_status = scraper.scrape_url(
    #     url,
    #     formats=['markdown']
    # )
    # return scrape_status.markdown
    result = fetching_description_from_huggingface(url)
    return result

In [574]:
class WebScraperParameters(BaseModel):
    url: str

In [575]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [576]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [577]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY
)

In [578]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )

In [579]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [580]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
                    You are an expert in task formulation. Given a machine learning task, you will
                    help divide the task into these subtasks:
                    - Subtask1: for classifying the problem type and task description
                    - Subtask2: for identifying the input of the problem
                    - Subtask3: for identifying the output of the problem
                    describe the each subtask as specifically as possible
                    """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [581]:
state = {
    "task": HumanMessage(content=task),
}

In [582]:
state = classify_agent(state)

# Modeling Agent

In [583]:
class ModelInfo(BaseModel):
    model: str = Field(..., description="Model name")
    url: str = Field(..., description="URL to the model")

class ModelSelection(BaseModel):
    models: List[ModelInfo] = Field(..., description="List of available models with name and URL")

In [584]:
model_selection_parser = PydanticOutputParser(pydantic_object=ModelSelection)

In [585]:
def model_selection_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the suitable model for a given task.
                    Given:
                    - A user description about the task
                    - A list of available model (names and links)
                    Your job is to choose ALL suitable models provided in the list for user's specific tasks.
                    Return the answer in the format
                    {structured_output}
                    Here are the list of model:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}")
    ]).partial(structured_output=model_selection_parser.get_format_instructions())

    chain = prompt | llm | model_selection_parser

    result = chain.invoke({"input": task, "model_list": format_model_list(df_model)})
    return result

In [586]:
res = model_selection_agent(state)

/tmp/ipykernel_27136/7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
/tmp/ipykernel_27136/7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])


In [587]:
pprint.pprint(res.models)

[ModelInfo(model='Nhận diện ảnh chữ viết tay', url='https://huggingface.co/thanhtlx/image_classification_01')]


In [588]:
def model_selection_using_tool_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the **Best-fit model** for a given task.
                    Given:
                    - A user task description
                    - A list of available models (names and url)
                    - A tool that fetches model details from a provided URL (it help gathers model description, input/output format, code sample usage)

                    Your job is to:
                    1. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🛠️ Library requirements
                        - 🧪 Code sample (usage code snippet)
                    2. Select the best model for user given task based on the data that the tool gives you.

                    ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🛠️ Library Requirements (from tool) **
                    <requirements to use the model>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>

                    Here are the list of models and there corresponding URL:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    # result = executor.invoke({"input": task_input})
    result = executor.invoke({"input": task_input, "model_list": res.models})

    return {"output": result["output"]}

In [589]:
res2 = model_selection_using_tool_agent(state)



> Entering new AgentExecutor chain...



Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/thanhtlx/image_classification_01'}`
responded: I will use the provided tool to gather information on this model. Let's start.




Image Classification Model - Handwritten Digit Recognition (Color Images)
📝 Mô tả
Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số viết tay (từ 0 đến 9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision Transformer (ViT) và được huấn luyện với tập dữ liệu gồm các chữ số viết tay được chụp hoặc scan dưới dạng ảnh màu.
📌 Chi tiết các mô hình
Phân loại ảnh chữ số viết tay từ 0 đến 9
📥 Đầu vào
Định dạng: Ảnh màu (RGB)
Kiểu dữ liệu: PIL.Image.Image
Kích thước ảnh: Mô hình sẽ tự resize theo cấu hình của ViTImageProcessor (thường là 224x224)
Giá trị pixel: [0, 255] (chuẩn hóa tự động bởi processor)
📤 Đầu ra
Một số nguyên từ 0 đến 9, là kết quả phân loại tương ứng với chữ số xuất hiện trong ảnh.
🛠 Yêu cầu thư viện
Cài đặt các thư viện sau bằng pip:
pip install transformers torch
🧪 

In [590]:
pprint.pprint(res2['output'])

('The model provided is for handwritten digit recognition, which can classify '
 'handwritten digits from 0 to 9. However, the task requires not only '
 'recognizing the digit but also determining if the recognized digit is a '
 'prime number or not.\n'
 '\n'
 "The model's output is a single digit from 0 to 9. To determine if this digit "
 'is prime, we can use a simple mathematical function to check for primality.\n'
 '\n'
 "Given the model's capabilities and the task's requirements, we can use the "
 'model as is for digit recognition and then apply a primality test on the '
 'recognized digit.\n'
 '\n'
 'Here is the final answer:\n'
 '\n'
 ' **✅ Model name and link**  \n'
 ' `Nhận diện ảnh chữ viết tay` — '
 '`https://huggingface.co/thanhtlx/image_classification_01`\n'
 '\n'
 ' **📝 Description (from tool):**  \n'
 ' Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số '
 'viết tay (từ0 đến9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision '
 'Transformer (ViT) và

## Output Parser Agent

In [602]:
class ModelingOutput(BaseModel):
    model_description: str = Field(
        ...,
        description="The detailed description of the model."
    )
    model_input_format: str = Field(
        ...,
        description="Model's detailed input format"
    )
    model_output_format: str = Field(
        ...,
        description="Model's detailed output format"
    )
    model_requirements: str = Field(
        ...,
        description="Requirement libraries need to install to use the model."
    )
    model_sample_code: str = Field(
        ...,
        description="Example code to use the model"
    )

In [604]:
parser = PydanticOutputParser(pydantic_object=ModelingOutput)

In [605]:
output_parser_llm = ChatOpenAI(
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0
)

In [606]:
def output_parser_agent(state: AgentState):
    system_prompt = """
                    You are a smart AI tasked with extracting structured technical details about a machine learning model from a reasoning result.
                    You are given a detailed text description about a model.
                    Your goal is to fill the following fields based on the text:

                    - `model_description`: A detailed explanation of what the model is and what it does.
                    - `model_input_format`: A detailed description of the model's input format, including dimensions, data types, and expected preprocessing if mentioned.
                    - `model_output_format`: A detailed description of the output format including dimensions, data types, label name and its meaning.
                    - `model_requirements`: A detailed description about the requirements needed to be sastified in order to use the model
                    - `model_sample_code`: A sample code on how to use the model.

                    You MUSTN'T create any data on your own, only using the data provided in the text.

                    Return the data as a JSON object matching the following structure:
                    {formatted_output}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        ('human', "{input}")
    ]).partial(formatted_output=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"input": res2['output']})
    return response

In [607]:
ans = output_parser_agent(state)

In [608]:
pprint.pprint(res2['output'])

('The model provided is for handwritten digit recognition, which can classify '
 'handwritten digits from 0 to 9. However, the task requires not only '
 'recognizing the digit but also determining if the recognized digit is a '
 'prime number or not.\n'
 '\n'
 "The model's output is a single digit from 0 to 9. To determine if this digit "
 'is prime, we can use a simple mathematical function to check for primality.\n'
 '\n'
 "Given the model's capabilities and the task's requirements, we can use the "
 'model as is for digit recognition and then apply a primality test on the '
 'recognized digit.\n'
 '\n'
 'Here is the final answer:\n'
 '\n'
 ' **✅ Model name and link**  \n'
 ' `Nhận diện ảnh chữ viết tay` — '
 '`https://huggingface.co/thanhtlx/image_classification_01`\n'
 '\n'
 ' **📝 Description (from tool):**  \n'
 ' Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số '
 'viết tay (từ0 đến9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision '
 'Transformer (ViT) và

In [610]:
pprint.pprint(ans.model_description)
pprint.pprint(ans.model_input_format)
pprint.pprint(ans.model_output_format)
pprint.pprint(ans.model_requirements)
pprint.pprint(ans.model_sample_code)

('This is a handwritten digit recognition model that can classify handwritten '
 'digits from 0 to 9. The model uses a Vision Transformer (ViT) architecture '
 'and was trained on a dataset of handwritten digits. The task requires not '
 'only recognizing the digit but also determining if the recognized digit is a '
 'prime number or not.')
("The model's input format is a color image (RGB) with a PIL.Image.Image data "
 'type. The image size is automatically resized to 224x224 by the '
 'ViTImageProcessor. The pixel values range from [0,255] and are normalized '
 'automatically by the processor.')
("The model's output format is a single integer from 0 to 9, representing the "
 'classified digit. To determine if the digit is prime, a simple mathematical '
 'function can be applied to the output.')
('The required libraries to use the model are transformers and torch, which '
 'can be installed using pip: pip install transformers torch')
('```python\n'
 'from transformers import ViTForIma